<b>PROYECTO INTEGRADOR HENRY <b>

Agustin Agulleiro

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter

In [ ]:
#IMPORTO EL CSV
amazon = pd.read_csv("plataformas/amazon_prime_titles.csv")
amazon.head()

In [ ]:
#CREO UNA COLUMNA "ID" con la primera inicial de la plataforma y lo concateno con su "show_id"
amazon["id"] = "a" + amazon["show_id"]

In [ ]:
#DROPEAMOS LA COLUMNA "SHOW_ID"
amazon = amazon.drop(columns="show_id")

In [ ]:
#me fijo las columnas que tiene, para luego reindexarlas 
amazon.columns

In [ ]:
#cambio las columnas de lugar, para que "id" este primero
amazon = amazon.reindex(columns=["id",'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description'])

In [ ]:
#Miramos la cantidad de nulos por columna
amazon.isnull().sum()

In [ ]:
#REMPLAZAMOS LOS NULOS DE RATING POR LA LETRA "G"
amazon["rating"] = amazon["rating"].fillna("G")

In [ ]:
# CONVERTIR LA COLUMNA DE STRING A FORMATO DE FECHA
amazon['date_added'] = pd.to_datetime(amazon["date_added"])
# CONVERTIR EL FORMATO DE FECHA A "AAAA-MM-DD
amazon['date_added'] = amazon['date_added'].dt.strftime('%Y-%m-%d')

In [ ]:
# CONVERTIMOS TODAS LAS LETRAS DEL DATAFRAME A MINÚSCULAS
amazon = amazon.applymap(lambda s: s.lower() if type(s) == str else s)

In [ ]:
# DIVIDIR LA COLUMNA "DURATION" EN UN NÚMERO Y UN STRING Y CREAR LAS COLUMNAS CON SUS RESPECTIVAS VARIABLES RESULTADO
amazon[['duration_int', 'duration_type']] = amazon['duration'].str.split(' ', expand=True)
amazon.drop(columns=["duration"],inplace= True)

In [ ]:
#RELLENAMOS LOS NULOS COMO 0
amazon["duration_int"].fillna(0,inplace = True)
#CAMBIAMOS EL TIPO DE DATO A ENTERO EN LA TABLA DURATION_INT
amazon['duration_int'] = amazon['duration_int'].astype(int)

In [ ]:
#Cambiamos el formato a DATETIME
amazon["date_added"] = pd.to_datetime(amazon["date_added"])

<b>APARTIR DE ACA REALIZAMOS EXACTAMENTE LOS MISMOS PASOS EN LAS DEMAS PLATAFORMAS <b>

In [ ]:
# APLICAMOS LO REALIZADO ANTERIORMENTE CON LOS DEMAS CSV
disney = pd.read_csv("plataformas/disney_plus_titles.csv")
hulu = pd.read_csv("plataformas/hulu_titles.csv")
netflix = pd.read_csv("plataformas/netflix_titles.csv")
def transformacion (plataforma,letra):
    plataforma["id"] = letra + plataforma["show_id"]
    plataforma = plataforma.drop(columns="show_id")
    plataforma = plataforma.reindex(columns=["id",'type', 'title', 'director', 'cast', 'country', 'date_added',
                                    'release_year', 'rating', 'duration', 'listed_in', 'description'])
    plataforma["rating"] = plataforma["rating"].fillna("G")
    plataforma['date_added'] = pd.to_datetime(plataforma["date_added"])
    plataforma['date_added'] = plataforma['date_added'].dt.strftime('%Y-%m-%d')
    plataforma = plataforma.applymap(lambda s: s.lower() if type(s) == str else s)
    plataforma[['duration_int', 'duration_type']] = plataforma['duration'].str.split(' ', expand=True)
    plataforma.drop(columns=["duration"],inplace= True)
    plataforma["duration_int"].fillna(0,inplace = True)
    plataforma['duration_int'] = plataforma['duration_int'].astype(int)
    plataforma["date_added"] = pd.to_datetime(plataforma["date_added"])
    return plataforma

In [ ]:
#APLICAMOS EL MISMO PROCESO PARA LOS DEMAS CSV
disney = transformacion(disney, "d")
hulu = transformacion(hulu, "h")
netflix = transformacion(netflix, "n")

In [ ]:
# CONCATENAMOS TODOS LOS DATAFRAME EN UNO SOLO 
plataformas = pd.concat([amazon,disney,hulu,netflix])

In [ ]:
# CONVIERTO LA COLUMNA "date_added" a DATETIME
plataformas["date_added"] = pd.to_datetime(plataformas["date_added"])

<b>TRANSFORMAMOS LOS CSV DE RATING<B>

In [ ]:
#IMPORTAMOS TODOS LOS CSV EN UNA VARIABLE DIFERENTE
rating1 = pd.read_csv("ratings/1.csv")
rating2 = pd.read_csv("ratings/2.csv")
rating3 = pd.read_csv("ratings/3.csv")
rating4 = pd.read_csv("ratings/4.csv")
rating5 = pd.read_csv("ratings/5.csv")
rating6 = pd.read_csv("ratings/6.csv")
rating7 = pd.read_csv("ratings/7.csv")
rating8 = pd.read_csv("ratings/8.csv")

In [ ]:
#UNIMOS TODOS LAS VARIABLES EN UNA SOLA CON UN CONCAT(CONCATENAR)
df_rating = pd.concat([rating1,rating2,rating3,rating4,rating5,rating6,rating7,rating8])

In [ ]:
#AGRUPAMOS TODAS LAS VARIABLES REPETIDAS EN "MOVIEID".
#PARA QUE NO HAYA REPETICION DE INFORMACION, SACAMOS UN PROMEDIO Y LO RENDONDEAMOS
df_agrupado = round(df_rating.groupby('movieId', as_index=False)['rating'].mean())

In [ ]:
#TRANSFORMAMOS LA COLUMNA RATING A TIPO ENTERO (ESTABA EN FLOAT)
df_agrupado["rating"] = df_agrupado["rating"].astype(int)

In [ ]:
plataformas = pd.merge(plataformas , df_agrupado, left_on='id',right_on='movieId')

In [ ]:
plataformas = plataformas.drop(columns="movieId")

In [ ]:
# GUARDAMOS EL DATAFRAME EN UN CSV
plataformas.to_csv("plataformas.csv",index=False)

<b>CREO LAS FUNCIONES PARA IMPLEMENTARLAS EN LA API <b>

In [60]:
def get_max_duration(year: int, platform: str, duration_type: str):
    if not isinstance(year, int):
        raise ValueError("El valor de 'year' debe ser un número entero")
    if not isinstance(platform, str):
        raise ValueError("El valor de 'platform' debe ser una cadena de caracteres")
    if not isinstance(duration_type, str):
        raise ValueError("El valor de 'duration_type' debe ser una cadena de caracteres")
    df = pd.read_csv("plataformas.csv")
    if platform[0] in ["h", "n", "a", "d", "H", "N", "A", "D"]:
        platform = platform.lower()[0]
        duration_type = duration_type.lower()
        if duration_type == "min":
            resultado = df[(df['release_year']==year) & (df['id'].str.startswith(platform)) & (df['duration_type']== duration_type) & (df["type"] == "movie")]
            idx = resultado['duration_int'].idxmax()
            return {resultado.loc[idx, 'title']}
        else:
            return {"Para saber cual fue la pelicula maxima trata de poner en duration_type min"}
    else: 
        return {"Intente usar una plataforma existente"}

In [59]:
def get_score_count(platform: str, scored: int, year: int):
    df = pd.read_csv("plataformas.csv")
    if platform[0] in ["h", "n", "a", "d", "H", "N", "A", "D"]:
        platform = platform.lower()[0]
        respuesta = df[(df['id'].str.startswith(platform)) & (df["rating_y"]> scored) & (df["release_year"]== year) & (df["type"]== "movie")]
        cantidad = respuesta.shape[0]
        return {cantidad}
    else: 
        return {"Intente usar una plataforma existente"}

In [58]:
def get_count_platform(platform: str):
    df = pd.read_csv("plataformas.csv")
    if platform in ["amazon", "netflix", "hulu", "disney"]:
        platform = platform.lower()[0]
        respuesta = df[(df["id"].str.startswith(platform)) & (df["type"]== "movie")]
        cantidad = respuesta.shape[0]
        return {cantidad}
    else: 
        return {"Intente usar una plataforma existente"}

In [57]:
def get_actor(platform: str, year: int):
    if not isinstance(platform, str):
        raise ValueError("El valor de 'platform' debe ser de caracteres")
    if not isinstance(year, int):
        raise ValueError("El valor de 'year' debe ser un entero")
    df = pd.read_csv("plataformas.csv")
    if platform[0] in ["h", "n", "a", "d", "H", "N", "A", "D"]:
        platform1 = platform
        platform = platform.lower()[0]
        df = df[(df['id'].str.startswith(platform)) & (df['release_year'] == year)]
        actores_por_fila = df['cast'].dropna().apply(lambda x: [i.strip() for i in x.split(',') if i.strip() and not i.strip().isdigit()])
        contador_actores = Counter()
        for actores in actores_por_fila:
            contador_actores.update(actores)
        actor_mas_repetido = contador_actores.most_common(1)[0][0]
        cantidad_actor_mas_repetido = contador_actores[actor_mas_repetido]
        return f"El actor que aparece más veces es {actor_mas_repetido}, con {cantidad_actor_mas_repetido} apariciones, en el año {year} de la plataforma {platform1}"
    else:
        return f"Los valores ingresados son incorrectos intente nuevamente, recuerde que las plataformas son Netflix, Amazon, Disney o Hulu."

In [56]:
def prod_per_county(tipo: str,pais: str,anio: int):
    if not isinstance(tipo, str):
        raise ValueError("El valor de 'tipo' debe ser de caracteres")
    if not isinstance(pais, str):
        raise ValueError("El valor de 'platform' debe ser una cadena de caracteres")
    if not isinstance(anio, int):
        raise ValueError("El valor de 'duration_type' debe ser un entero")
    df = pd.read_csv("plataformas.csv")
    tipo = tipo.lower()
    pais = pais.lower()
    if tipo == "tv show" or tipo == "movie":
        if pais in df["country"].unique():
            respuesta = df[(df["type"]== tipo) & (df["country"]== pais) & (df["release_year"]== anio)].shape[0]
            return {'pais': pais,
                    'anio': anio,
                    'peliculas': respuesta}
        else:
            return {f"Intenta poner un pais correcto"}
    else:
        return {f"Intenta poner tv show o movie en lugar de {tipo}"}

In [68]:
def get_contents(rating: str):
    df = pd.read_csv("plataformas.csv")
    respuesta = df[df["rating_x"]== rating]
    return {respuesta.shape[0]}